# Installation

For installation instructions please refer to `Simply laced diagrams` notebook.

In [1]:
using Pkg

In [2]:
Pkg.activate(joinpath(@__DIR__, ".."))

  Activating project at `~/Mathematics/Research/Property (T)/Chevalley/2306.12358`


In [3]:
versioninfo()

Julia Version 1.9.1
Commit 147bdf428cd (2023-06-07 08:27 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 PRO 4750U with Radeon Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, znver2)
  Threads: 9 on 16 virtual cores
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_IMAGE_THREADS = 1


In [4]:
Pkg.status()

Status `~/Mathematics/Research/Property (T)/Chevalley/2306.12358/Project.toml`
  [1e616198] COSMO v0.8.7
  [5d8bd718] Groups v0.7.7
  [7073ff75] IJulia v1.24.2
  [4076af6c] JuMP v1.12.0
  [03b72c93] PropertyT v0.5.0 `https://github.com/kalmarek/PropertyT.jl#master`
  [c946c3f1] SCS v1.2.1
⌅ [856f044c] MKL_jll v2022.2.0+0 ⚲
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [5]:
using Groups
import Groups.MatrixGroups

In [6]:
using PropertyT
using PropertyT.IntervalArithmetic

> In this notebook only rudimentary commentary is included. For the extended one please consult `Simply laced diagrams` notebook.

# Sum of squares proof of property (T) for $\operatorname{Sp}_{4}(\mathbb{Z})$

We wish to prove
> **Theorem 3.12** Let $G= \operatorname{Sp}_{4}(\mathbb{Z})$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{B}_{\texttt{2}} = \texttt{C}_{\texttt{2}}$ endowed with the set of Steinberg generators $S$. The pair $(G, S)$ has property (T) with a witness of type $(\lambda, R) = (0.87890, 2)$.

We will show this by exhibiting $\xi_i\in \mathbb{R}G$, supported inside $\operatorname{Ball}(S, 2)$ such that

$$
\Delta^2 - \lambda \Delta - \sum_i \xi_i^* \xi_i = r,
$$

with $\|r\|_1$ much smaller (a few orders of magnitue) than $\lambda$.

In [7]:
N = 2
G = MatrixGroups.SymplecticGroup{2*N}(Int8)

group of 4×4 symplectic matrices

In [8]:
gens(G, 1)

A₁.₂ ∈ Sp{4,Int8}
 1  1   0  0
 0  1   0  0
 0  0   1  0
 0  0  -1  1

In [9]:
gens(G, 3)

B₃.₁ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 1  0  1  0
 0  0  0  1

In [10]:
gens(G, 5)

B₃.₂ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 0  1  1  0
 1  0  0  1

In [11]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

  0.047663 seconds (462.15 k allocations: 18.142 MiB, 40.45% gc time)
  0.005926 seconds (72.67 k allocations: 2.805 MiB)
  1.033321 seconds (1.62 M allocations: 93.844 MiB, 3.94% gc time, 94.65% compilation time: 14% of which was recompilation)


[ Info: generating wl-metric ball of radius 4
[ Info: sizes = [17, 209, 2073, 18313]
[ Info: computing the *-algebra structure for G


In [12]:
S

16-element Vector{FPGroupElement{Groups.MatrixGroups.SymplecticGroup{4, Int8, DataType, Groups.MatrixGroups.ElementarySymplectic{4, Int8}}, …}}:
 A₁.₂
 A₂.₁
 B₃.₁
 B₄.₂
 B₃.₂
 B₁.₃
 B₂.₄
 B₂.₃
 A₁.₂^-1
 A₂.₁^-1
 B₃.₁^-1
 B₄.₂^-1
 B₃.₂^-1
 B₁.₃^-1
 B₂.₄^-1
 B₂.₃^-1

In [13]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

16·(id) -1·A₁.₂ -1·A₂.₁ -1·B₃.₁ -1·B₄.₂ -1·B₃.₂ -1·B₁.₃ -1·B₂.₄ -1·B₂.₃ -1·A₁.₂^-1 -1·A₂.₁^-1 -1·B₃.₁^-1 -1·B₄.₂^-1 -1·B₃.₂^-1 -1·B₁.₃^-1 -1·B₂.₄^-1 -1·B₂.₃^-1

## Optimization problem

In [14]:
@time model = PropertyT.sos_problem_primal(
    Δ^2,
    Δ,
    augmented = true,
)

  2.809328 seconds (6.76 M allocations: 447.574 MiB, 8.29% gc time, 91.02% compilation time: <1% of which was recompilation)


A JuMP Model
Maximization problem with:
Variables: 21946
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 18313 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: P, psd, λ

However, since we know the action of the Weyl group, we do not want to solve this formulation.

### Symmetry reduction

In [15]:
import PropertyT.StarAlgebras
import PropertyT.SymbolicWedderburn
using PropertyT.PermutationGroups

In [16]:
wd = let RG = RG, N = N
    G = StarAlgebras.object(RG)
    P = PermGroup(perm"(1,2)", Perm(circshift(1:N, -1)))
    Weyl = Groups.Constructions.WreathProduct(PermGroup(perm"(1,2)"), P)
    act = PropertyT.action_by_conjugation(G, Weyl)

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Weyl,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
    )
end
@info wd

 10.315591 seconds (15.28 M allocations: 996.795 MiB, 5.75% gc time, 180.51% compilation time)


┌ Info: Wedderburn Decomposition into 4830 orbits and 4 simple summands of sizes
└ [71, 58, 44, 36]


In [17]:
@time model, varP = PropertyT.sos_problem_primal(Δ^2, Δ, wd; augmented = true);
model

  3.145996 seconds (4.65 M allocations: 544.512 MiB, 7.48% gc time, 207.87% compilation time)


A JuMP Model
Maximization problem with:
Variables: 5924
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 4829 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

In [18]:
include(joinpath(dirname(pathof(PropertyT)), "..", "test", "optimizers.jl"));
with_optimizer = cosmo_optimizer(;
    eps = 1e-9,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.95,
);

In [19]:
warm = nothing

In [20]:
status, warm = PropertyT.solve(model, with_optimizer, warm)
@info "Optimization has finished with" status

------------------------------------------------------------------
          COSMO v0.8.7 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{5924},
          constraints: A ∈ R^{10752x5924} (46608 nnz),
          matrix size to factor: 16676x16676,
          Floating-point precision: Float64
Sets:     ZeroSet of dim: 4829
          PsdConeTriangle of dim: 2556 (71x71)
          PsdConeTriangle of dim: 1711 (58x58)
          PsdConeTriangle of dim: 990 (44x44)
          PsdConeTriangle of dim: 666 (36x36)
          ... and 0 more
Settings: ϵ_abs = 1.0e-09, ϵ_rel = 1.0e-09,
          ϵ_prim_inf = 1.0e-09, ϵ_dual_inf = 1.0e-09,
          ρ = 0.1, σ = 1e-06, α = 1.95,
          max_iter = 20000,
          scaling iter = 10 (on),
          check termination every 250 iter,
          check infeasibility every 40 iter,
          KKT 

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


### Reconstructing and certifying the solution
Since we symmetrized our problem, we don't have direct access to `P` or `Q` as defined above. So first we need to reconstruct `Q` from the action of `Weyl` on $\operatorname{Ball}(S, R)$:

In [21]:
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

 12.127439 seconds (21.09 M allocations: 1.321 GiB, 5.39% gc time, 101.81% compilation time)


209×209 Matrix{Float64}:
 0.0   0.0          0.0          0.0         …   0.0           0.0
 0.0   2.32376      0.240807     0.149926       -0.00871198    0.0146467
 0.0   0.240807     2.32376     -0.0270739      -0.00871198    0.0146467
 0.0   0.149926    -0.0270739    2.92568        -0.0101338    -0.00672878
 0.0  -0.0270739    0.149926     0.103023       -0.0101338    -0.00672878
 0.0   0.068979     0.068979    -0.0270739   …  -0.00898872    0.00720544
 0.0  -0.0270739    0.149926    -0.0792421       0.0298764     0.0692956
 0.0   0.149926    -0.0270739    0.1528          0.0298764     0.0692956
 0.0   0.068979     0.068979     0.149926       -0.00517651    0.00692669
 0.0   0.184117    -0.0699968    0.149926       -0.00871198    0.0146467
 0.0  -0.0699968    0.184117    -0.0270739   …  -0.00871198    0.0146467
 0.0   0.149926    -0.0270739    0.231366       -0.0101338    -0.00672876
 0.0  -0.0270739    0.149926     0.1528         -0.0101338    -0.00672876
 ⋮                        

In [22]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    Δ^2,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Sp_{$(2N)}(ℤ) has property (T)" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the Property(T) for Sp_{$(2N)}(ℤ)" certified λ
end

[ Info: certifying the solution


  0.001106 seconds (6 allocations: 484.656 KiB)


┌ Info: Checking in Float64 arithmetic with
└   λ = 0.8791635509146072
┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ -9.539847720347621e-13
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 9.74471207144253e-7
└  λ ≈ 0.8791596530297786


  0.672544 seconds (815.49 k allocations: 62.067 MiB, 7.64% gc time, 98.45% compilation time)


┌ Info: Checking in Interval{Float64} arithmetic with
└   λ_int = [0.879163, 0.879164]
┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ∈ [-7.1263e-10, 7.2096e-10]
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ∈ [9.73749e-07, 9.75184e-07]
└  λ ∈ [0.879159, 0.87916]
┌ Info: Certified result: Sp_{4}(ℤ) has property (T)
└   PropertyT.IntervalArithmetic.inf(λ) = 0.8791596501814338


# Levels in $\operatorname{Sp}_{6}(\mathbb{Z})$

We wish here to prove
> **Theorem 3.15**: Let $G= \operatorname{Sp}_{6}(\mathbb{Z})$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{C}_\texttt{3}$ endowed with the set of Steinberg generators $S$. Let $V$ denote the ambient vector space of the root system. Then for $\lambda=2.41212$ we have
>
>$$ \operatorname{Lev}_2^3 + \operatorname{Lev}_3^3 -\lambda \Delta_V = \sum \xi_i^* \xi_i$$
>
>for some $\xi_i\in \mathbb{R}G$ supported inside $\operatorname{Ball}(S, 2)$.

In [23]:
N = 3
G = MatrixGroups.SymplecticGroup{2*N}(Int8)

group of 6×6 symplectic matrices

In [24]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

[ Info: generating wl-metric ball of radius 4


  1.489715 seconds (12.08 M allocations: 598.526 MiB, 20.44% gc time)


[ Info: sizes = [37, 961, 21183, 424899]
[ Info: computing the *-algebra structure for G


  0.220843 seconds (1.69 M allocations: 94.096 MiB, 17.50% gc time)
  2.614683 seconds (14.93 M allocations: 768.927 MiB, 14.56% gc time, 34.55% compilation time)


In [25]:
S

36-element Vector{FPGroupElement{Groups.MatrixGroups.SymplecticGroup{6, Int8, DataType, Groups.MatrixGroups.ElementarySymplectic{6, Int8}}, …}}:
 A₁.₂
 A₁.₃
 A₂.₁
 A₂.₃
 A₃.₁
 A₃.₂
 B₄.₁
 B₅.₂
 B₆.₃
 B₄.₂
 B₄.₃
 B₅.₃
 B₁.₄
 ⋮
 B₄.₁^-1
 B₅.₂^-1
 B₆.₃^-1
 B₄.₂^-1
 B₄.₃^-1
 B₅.₃^-1
 B₁.₄^-1
 B₂.₅^-1
 B₃.₆^-1
 B₂.₄^-1
 B₃.₄^-1
 B₃.₅^-1

In [26]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

36·(id) -1·A₁.₂ -1·A₁.₃ -1·A₂.₁ -1·A₂.₃ -1·A₃.₁ -1·A₃.₂ -1·B₄.₁ -1·B₅.₂ -1·B₆.₃ -1·B₄.₂ -1·B₄.₃ -1·B₅.₃ -1·B₁.₄ -1·B₂.₅ -1·B₃.₆ -1·B₂.₄ -1·B₃.₄ -1·B₃.₅ -1·A₁.₂^-1 -1·A₁.₃^-1 -1·A₂.₁^-1 -1·A₂.₃^-1 -1·A₃.₁^-1 -1·A₃.₂^-1 -1·B₄.₁^-1 -1·B₅.₂^-1 -1·B₆.₃^-1 -1·B₄.₂^-1 -1·B₄.₃^-1 -1·B₅.₃^-1 -1·B₁.₄^-1 -1·B₂.₅^-1 -1·B₃.₆^-1 -1·B₂.₄^-1 -1·B₃.₄^-1 -1·B₃.₅^-1

## Grading

Let us define a $\texttt{C}_\texttt{2}$ grading for $(G, S)$. That is, to every generator $s \in S$ we need to assign a root in $\texttt{C}_\texttt{2}$ in a compatible fashion. For a precise description of this grading see **Example 3.2 (2)**.  

In [27]:
g = gens(G, 1)

A₁.₂ ∈ Sp{6,Int8}
 1  1  0   0  0  0
 0  1  0   0  0  0
 0  0  1   0  0  0
 0  0  0   1  0  0
 0  0  0  -1  1  0
 0  0  0   0  0  1

In [28]:
PropertyT.grading(g)

Root in ℝ^3 of length √2
[1, -1, 0]

In [29]:
g = gens(G, 7)

B₄.₁ ∈ Sp{6,Int8}
 1  0  0  0  0  0
 0  1  0  0  0  0
 0  0  1  0  0  0
 1  0  0  1  0  0
 0  0  0  0  1  0
 0  0  0  0  0  1

In [30]:
PropertyT.grading(g)

Root in ℝ^3 of length 2
[-2, 0, 0]

In [31]:
g = gens(G, 10)

B₄.₂ ∈ Sp{6,Int8}
 1  0  0  0  0  0
 0  1  0  0  0  0
 0  0  1  0  0  0
 0  1  0  1  0  0
 1  0  0  0  1  0
 0  0  0  0  0  1

In [32]:
PropertyT.grading(g)

Root in ℝ^3 of length √2
[-1, -1, 0]

In [33]:
Δs = PropertyT.laplacians(
    RG,
    S,
    x -> (gx = PropertyT.grading(x); Set([gx, -gx])),
);

According to **Definition 3.13** we define $\operatorname{Lev}_2^3$ as follows (note: `PropertyT.Adj(Δs, :A₁)` stands here for short squares).

In [34]:
Lev₂³ = PropertyT.Adj(Δs, :A₁) + PropertyT.Adj(Δs, :C₂)

696·(id) -32·A₁.₂ -32·A₁.₃ -32·A₂.₁ -32·A₂.₃ -32·A₃.₁ -32·A₃.₂ -48·B₄.₁ -48·B₅.₂ -48·B₆.₃ -32·B₄.₂ -32·B₄.₃ -32·B₅.₃ -48·B₁.₄ -48·B₂.₅ -48·B₃.₆ -32·B₂.₄ -32·B₃.₄ -32·B₃.₅ -32·A₁.₂^-1 -32·A₁.₃^-1 -32·A₂.₁^-1 -32·A₂.₃^-1 -32·A₃.₁^-1 -32·A₃.₂^-1 -48·B₄.₁^-1 -48·B₅.₂^-1 -48·B₆.₃^-1 -32·B₄.₂^-1 -32·B₄.₃^-1 -32·B₅.₃^-1 -48·B₁.₄^-1 -48·B₂.₅^-1 -48·B₃.₆^-1 -32·B₂.₄^-1 -32·B₃.₄^-1 -32·B₃.₅^-1 +1·A₁.₂^2 +1·A₁.₂*A₂.₁ +1·A₁.₂*B₄.₁ +2·A₁.₂*B₅.₂ +1·A₁.₂*B₄.₂ +2·A₁.₂*B₁.₄ +1·A₁.₂*B₂.₅ +1·A₁.₂*B₂.₄ +1·A₁.₂*A₂.₁^-1 +1·A₁.₂*B₄.₁^-1 +2·A₁.₂*B₅.₂^-1 +1·A₁.₂*B₄.₂^-1 +2·A₁.₂*B₁.₄^-1 +1·A₁.₂*B₂.₅^-1 +1·A₁.₂*B₂.₄^-1 +1·A₁.₃^2 +1·A₁.₃*A₃.₁ +1·A₁.₃*B₄.₁ +2·A₁.₃*B₆.₃ +1·A₁.₃*B₄.₃ +2·A₁.₃*B₁.₄ +1·A₁.₃*B₃.₆ +1·A₁.₃*B₃.₄ +1·A₁.₃*A₃.₁^-1 +1·A₁.₃*B₄.₁^-1 +2·A₁.₃*B₆.₃^-1 +1·A₁.₃*B₄.₃^-1 +2·A₁.₃*B₁.₄^-1 +1·A₁.₃*B₃.₆^-1 +1·A₁.₃*B₃.₄^-1 +1·A₂.₁*A₁.₂ +1·A₂.₁^2 +2·A₂.₁*B₄.₁ +1·A₂.₁*B₅.₂ +1·A₂.₁*B₄.₂ +1·A₂.₁*B₁.₄ +2·A₂.₁*B₂.₅ +1·A₂.₁*B₂.₄ +1·A₂.₁*A₁.₂^-1 +2·A₂.₁*B₄.₁^-1 +1·A₂.₁*B₅.₂^-1 +1·A₂.₁*B₄.₂^-1 +1·A₂.₁*B₁.₄^-1 +2·A₂.

In [35]:
Lev₃³ = PropertyT.Adj(Δs, :A₂) + PropertyT.Adj(Δs, Symbol("A₁×C₁"))

576·(id) -40·A₁.₂ -40·A₁.₃ -40·A₂.₁ -40·A₂.₃ -40·A₃.₁ -40·A₃.₂ -16·B₄.₁ -16·B₅.₂ -16·B₆.₃ -40·B₄.₂ -40·B₄.₃ -40·B₅.₃ -16·B₁.₄ -16·B₂.₅ -16·B₃.₆ -40·B₂.₄ -40·B₃.₄ -40·B₃.₅ -40·A₁.₂^-1 -40·A₁.₃^-1 -40·A₂.₁^-1 -40·A₂.₃^-1 -40·A₃.₁^-1 -40·A₃.₂^-1 -16·B₄.₁^-1 -16·B₅.₂^-1 -16·B₆.₃^-1 -40·B₄.₂^-1 -40·B₄.₃^-1 -40·B₅.₃^-1 -16·B₁.₄^-1 -16·B₂.₅^-1 -16·B₃.₆^-1 -40·B₂.₄^-1 -40·B₃.₄^-1 -40·B₃.₅^-1 +2·A₁.₂*A₁.₃ +1·A₁.₂*A₂.₃ +1·A₁.₂*A₃.₁ +2·A₁.₂*A₃.₂ +2·A₁.₂*B₆.₃ +1·A₁.₂*B₄.₃ +2·A₁.₂*B₅.₃ +2·A₁.₂*B₃.₆ +2·A₁.₂*B₃.₄ +1·A₁.₂*B₃.₅ +2·A₁.₂*A₁.₃^-1 +1·A₁.₂*A₂.₃^-1 +1·A₁.₂*A₃.₁^-1 +2·A₁.₂*A₃.₂^-1 +2·A₁.₂*B₆.₃^-1 +1·A₁.₂*B₄.₃^-1 +2·A₁.₂*B₅.₃^-1 +2·A₁.₂*B₃.₆^-1 +2·A₁.₂*B₃.₄^-1 +1·A₁.₂*B₃.₅^-1 +1·A₁.₃*A₂.₁ +2·A₁.₃*A₂.₃ +1·A₁.₃*A₃.₂ +2·A₁.₃*B₅.₂ +1·A₁.₃*B₄.₂ +2·A₁.₃*B₅.₃ +2·A₁.₃*B₂.₅ +2·A₁.₃*B₂.₄ +1·A₁.₃*B₃.₅ +2·A₁.₃*A₁.₂^-1 +1·A₁.₃*A₂.₁^-1 +2·A₁.₃*A₂.₃^-1 +1·A₁.₃*A₃.₂^-1 +2·A₁.₃*B₅.₂^-1 +1·A₁.₃*B₄.₂^-1 +2·A₁.₃*B₅.₃^-1 +2·A₁.₃*B₂.₅^-1 +2·A₁.₃*B₂.₄^-1 +1·A₁.₃*B₃.₅^-1 +1·A₂.₁*A₁.₃ +2·A₂.₁*A₂.₃ +2·A₂.₁*A₃.₁ +1·A₂.₁

## Optimization problem

### Symmetry reduction

In [36]:
import PropertyT.StarAlgebras
import PropertyT.SymbolicWedderburn
using PropertyT.PermutationGroups

In [37]:
wd = let RG = RG, N = N
    G = StarAlgebras.object(RG)
    P = PermGroup(perm"(1,2)", Perm(circshift(1:N, -1)))
    Weyl = Groups.Constructions.WreathProduct(PermGroup(perm"(1,2)"), P)
    act = PropertyT.action_by_conjugation(G, Weyl)

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Weyl,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
    )
end
@info wd

  8.808766 seconds (23.77 M allocations: 1.483 GiB, 4.34% gc time, 155.27% compilation time)


┌ Info: Wedderburn Decomposition into 21968 orbits and 5 simple summands of sizes
└ [152, 97, 56, 94, 74]


In [38]:
@time model, varP = PropertyT.sos_problem_primal(
    Lev₂³ + Lev₃³,
    Δ,
    wd;
    augmented = true,
);
model

  6.510428 seconds (27.58 M allocations: 5.994 GiB, 11.15% gc time, 6.73% compilation time)


A JuMP Model
Maximization problem with:
Variables: 25218
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 21967 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

In [39]:
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));
with_optimizer = cosmo_optimizer(;
    eps = 1e-10,
    max_iters = 40_000,
    accel = 50,
    alpha = 1.6,
);

In [40]:
warm = nothing

> **Note**: Running the following optimization step may take a few minutes.

In [41]:
status, warm = PropertyT.solve(
        model,
        with_optimizer,
        warm,
    );
@info "Optimization has finished with" status

------------------------------------------------------------------
          COSMO v0.8.7 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{25218},
          constraints: A ∈ R^{47184x25218} (301568 nnz),
          matrix size to factor: 72402x72402,
          Floating-point precision: Float64
Sets:     ZeroSet of dim: 21967
          PsdConeTriangle of dim: 11628 (152x152)
          PsdConeTriangle of dim: 4753 (97x97)
          PsdConeTriangle of dim: 4465 (94x94)
          PsdConeTriangle of dim: 2775 (74x74)
          ... and 1 more
Settings: ϵ_abs = 1.0e-10, ϵ_rel = 1.0e-10,
          ϵ_prim_inf = 1.0e-10, ϵ_dual_inf = 1.0e-10,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 40000,
          scaling iter = 10 (on),
          check termination every 250 iter,
          check infeasibility every 40 iter,
      

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


### Reconstructing and certifying the solution
Since we symmetrized our problem, we don't have direct access to `P` or `Q` as defined above. So first we need to reconstruct `Q` from the action of `Weyl` on $\operatorname{Ball}(S, R)$:

In [42]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

[ Info: reconstructing the solution


  0.286583 seconds (649.89 k allocations: 164.348 MiB, 51.61% compilation time)


961×961 Matrix{Float64}:
 0.0   0.0           0.0           0.0         …   0.0           0.0
 0.0   4.11888       0.0611561    -0.11867        -0.00160495   -0.0063545
 0.0   0.0611561     4.11888       0.0902717      -0.00160495   -0.0063545
 0.0  -0.11867       0.0902717     4.11888         0.00166186   -0.003316
 0.0   0.0902717     0.0611555     0.0611561       0.0075698     0.00474679
 0.0   0.0902717    -0.11867       0.0611555   …   0.00166186   -0.003316
 0.0   0.0611555     0.0902717     0.0902717       0.0075698     0.00474679
 0.0   0.0861588     0.0861588    -0.0339782       0.0191616     0.00747337
 0.0  -0.0339782     0.0478766     0.0861588      -0.00939305   -0.0152719
 0.0   0.0478766    -0.0339782     0.0478766      -0.00939305   -0.0152719
 0.0  -0.00940645    0.0263671    -0.00940645  …  -0.00168451   -0.00488268
 0.0   0.0263671    -0.00940645    0.0743582      -0.00168451   -0.00488268
 0.0   0.0743582     0.0743582     0.0263671      -0.000908028  -0.00148491
 ⋮

In [43]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    Lev₂³ + Lev₃³,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Lev₂³ + Lev₃³ is positive" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the positivity of Lev₂³ + Lev₃³" certified λ
end

[ Info: certifying the solution


  0.018779 seconds (6 allocations: 10.288 MiB)


┌ Info: Checking in Float64 arithmetic with
└   λ = 2.4173985001872422
┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ 3.800168905952744e-10
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 1.2639490076802686e-6
└  λ ≈ 2.4173934443912115


  0.282798 seconds (78.27 k allocations: 180.041 MiB, 38.47% compilation time)


┌ Info: Checking in Interval{Float64} arithmetic with
└   λ_int = [2.41739, 2.4174]
┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ∈ [-1.04938e-07, 1.02458e-07]
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ∈ [1.16185e-06, 1.36926e-06]
└  λ ∈ [2.41739, 2.4174]
┌ Info: Certified result: Lev₂³ + Lev₃³ is positive
└   PropertyT.IntervalArithmetic.inf(λ) = 2.4173930231753773


# $\texttt{C}_\texttt{2}$-graded $\operatorname{Adj}$

We wish to prove
> **Theorem 3.10** Let $G = \operatorname{Sp}_4(\mathbb{Z})$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{C}_\texttt{2}$ endowed with the set of Steinberg generators $S$. Let $V$ denote the ambient vector space of the root system. Then $$\operatorname{Adj}_V −\lambda \Delta_V ⩾_R 0$$
whenever $(\lambda, R) \in (0.24493, 3)$.

Proving this theorem for $R = 3$ requires quite a bit of memory (`~16GB`) and quite a bit of patience.

In [44]:
N = 2
G = MatrixGroups.SymplecticGroup{2*N}(Int8)

group of 4×4 symplectic matrices

In [45]:
HALFRADIUS = 3
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

[ Info: generating wl-metric ball of radius 6


  3.947620 seconds (49.50 M allocations: 1.687 GiB, 13.21% gc time)


[ Info: sizes = [17, 209, 2073, 18313, 149449, 1148696]
[ Info: computing the *-algebra structure for G


  0.667488 seconds (6.80 M allocations: 250.047 MiB, 22.27% gc time)
  4.615985 seconds (56.30 M allocations: 1.931 GiB, 14.51% gc time)


In [46]:
S

16-element Vector{FPGroupElement{Groups.MatrixGroups.SymplecticGroup{4, Int8, DataType, Groups.MatrixGroups.ElementarySymplectic{4, Int8}}, …}}:
 A₁.₂
 A₂.₁
 B₃.₁
 B₄.₂
 B₃.₂
 B₁.₃
 B₂.₄
 B₂.₃
 A₁.₂^-1
 A₂.₁^-1
 B₃.₁^-1
 B₄.₂^-1
 B₃.₂^-1
 B₁.₃^-1
 B₂.₄^-1
 B₂.₃^-1

In [47]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

16·(id) -1·A₁.₂ -1·A₂.₁ -1·B₃.₁ -1·B₄.₂ -1·B₃.₂ -1·B₁.₃ -1·B₂.₄ -1·B₂.₃ -1·A₁.₂^-1 -1·A₂.₁^-1 -1·B₃.₁^-1 -1·B₄.₂^-1 -1·B₃.₂^-1 -1·B₁.₃^-1 -1·B₂.₄^-1 -1·B₂.₃^-1

## Grading

Let us define $\texttt{C}_\texttt{2}$ grading for $(G, S)$. That is, to every generator $s \in S$ we need to assign a root in $\texttt{C}_\texttt{2}$ in a compatible fashion. For the precise description of this grading see **Example 3.2 (2)**.  

In [48]:
using PropertyT.Roots

In [49]:
g = gens(G, 1)

A₁.₂ ∈ Sp{4,Int8}
 1  1   0  0
 0  1   0  0
 0  0   1  0
 0  0  -1  1

In [50]:
PropertyT.grading(g)

Root in ℝ^2 of length √2
[1, -1]

In [51]:
g = gens(G, 3)

B₃.₁ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 1  0  1  0
 0  0  0  1

In [52]:
PropertyT.grading(g)

Root in ℝ^2 of length 2
[-2, 0]

In [53]:
g = gens(G, 5)

B₃.₂ ∈ Sp{4,Int8}
 1  0  0  0
 0  1  0  0
 0  1  1  0
 1  0  0  1

In [54]:
PropertyT.grading(g)

Root in ℝ^2 of length √2
[-1, -1]

In [55]:
Δs = PropertyT.laplacians(
    RG,
    S,
    x -> (gx = PropertyT.grading(x); Set([gx, -gx])),
);

Here `Δs` is just a map from lines in the root system $\Omega = \texttt{C}_{\texttt{2}}$ to the corresponding Laplacians, e.g. below we can see that to the line through `α = [1, -1]` and `-α` (and the origin) we assign
$$ \Delta_{Lα} = 4 - A_{1,2} - A_{2,1} - A_{1,2}^{-1} - A_{2,1}^{-1}.$$ 

In [56]:
let α = Root([1,-1])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·A₁.₂ -1·A₂.₁ -1·A₁.₂^-1 -1·A₂.₁^-1

In [57]:
let α = Root([2,0])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·B₃.₁ -1·B₁.₃ -1·B₃.₁^-1 -1·B₁.₃^-1

In [58]:
let α = Root([-1,-1])
    Lα = Set([α, -α])
    Δs[Lα]
end

4·(id) -1·B₃.₂ -1·B₂.₃ -1·B₃.₂^-1 -1·B₂.₃^-1

Following the definition of $\operatorname{Adj}$ we define
$$ \operatorname{Adj}_{\texttt{C}_\texttt{2}} = 
\prod_{
    \langle L\alpha, L\beta \rangle \cap \Omega \cong \texttt{C}_{\texttt{2}}
} \Delta_{L\alpha} \Delta_{L\beta} $$

In [59]:
AdjC₂ = PropertyT.Adj(Δs, :C₂)

192·(id) -24·A₁.₂ -24·A₂.₁ -24·B₃.₁ -24·B₄.₂ -24·B₃.₂ -24·B₁.₃ -24·B₂.₄ -24·B₂.₃ -24·A₁.₂^-1 -24·A₂.₁^-1 -24·B₃.₁^-1 -24·B₄.₂^-1 -24·B₃.₂^-1 -24·B₁.₃^-1 -24·B₂.₄^-1 -24·B₂.₃^-1 +1·A₁.₂*B₃.₁ +2·A₁.₂*B₄.₂ +1·A₁.₂*B₃.₂ +2·A₁.₂*B₁.₃ +1·A₁.₂*B₂.₄ +1·A₁.₂*B₂.₃ +1·A₁.₂*B₃.₁^-1 +2·A₁.₂*B₄.₂^-1 +1·A₁.₂*B₃.₂^-1 +2·A₁.₂*B₁.₃^-1 +1·A₁.₂*B₂.₄^-1 +1·A₁.₂*B₂.₃^-1 +2·A₂.₁*B₃.₁ +1·A₂.₁*B₄.₂ +1·A₂.₁*B₃.₂ +1·A₂.₁*B₁.₃ +2·A₂.₁*B₂.₄ +1·A₂.₁*B₂.₃ +2·A₂.₁*B₃.₁^-1 +1·A₂.₁*B₄.₂^-1 +1·A₂.₁*B₃.₂^-1 +1·A₂.₁*B₁.₃^-1 +2·A₂.₁*B₂.₄^-1 +1·A₂.₁*B₂.₃^-1 +1·B₃.₁*A₁.₂ +2·B₃.₁*B₄.₂ +2·B₃.₁*B₃.₂ +2·B₃.₁*B₂.₄ +1·B₃.₁*B₂.₃ +1·B₃.₁*A₁.₂^-1 +2·B₃.₁*A₂.₁^-1 +2·B₃.₁*B₄.₂^-1 +2·B₃.₁*B₃.₂^-1 +2·B₃.₁*B₂.₄^-1 +1·B₃.₁*B₂.₃^-1 +1·B₄.₂*A₂.₁ +2·B₄.₂*B₃.₂ +2·B₄.₂*B₁.₃ +1·B₄.₂*B₂.₃ +2·B₄.₂*A₁.₂^-1 +1·B₄.₂*A₂.₁^-1 +2·B₄.₂*B₃.₁^-1 +2·B₄.₂*B₃.₂^-1 +2·B₄.₂*B₁.₃^-1 +1·B₄.₂*B₂.₃^-1 +1·B₃.₂*A₁.₂ +1·B₃.₂*A₂.₁ +1·B₃.₂*B₁.₃ +1·B₃.₂*B₂.₄ +1·B₃.₂*A₁.₂^-1 +1·B₃.₂*A₂.₁^-1 +2·B₃.₂*B₃.₁^-1 +2·B₃.₂*B₄.₂^-1 +1·B₃.₂*B₁.₃^-1 +1·B₃.₂*B₂.₄^-1 +1·B₁.₃*A₂.₁ +1·B₁

It is not hard to see that for $\Omega = \texttt{C}_{\texttt{2}}$ 
 * we are simply looking at products of all $\Delta_{L\alpha}$ and $\Delta_{L\beta}$ where $L\alpha \neq L\beta$, and
 * that the new definition is an analouge to the definition of $\operatorname{Adj}$ from [On property (T) for $\operatorname{Aut}(F_n)$ and $\operatorname{SL}_n(\mathbb{Z})$](https://arxiv.org/abs/1812.03456).

In [60]:
AdjC₂ == Δ^2 - sum(Δs[Lα]^2 for Lα in keys(Δs))

true

## Optimization problem

### Symmetry reduction

In [61]:
import PropertyT.StarAlgebras
import PropertyT.SymbolicWedderburn
using PropertyT.PermutationGroups

In [62]:
wd = let RG = RG, N = N
    G = StarAlgebras.object(RG)
    P = PermGroup(perm"(1,2)", Perm(circshift(1:N, -1)))
    Weyl = Groups.Constructions.WreathProduct(PermGroup(perm"(1,2)"), P)
    act = PropertyT.action_by_conjugation(G, Weyl)

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Weyl,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
    )
end
@info wd

  6.757325 seconds (52.04 M allocations: 2.952 GiB, 10.72% gc time, 10.59% compilation time: 72% of which was recompilation)


┌ Info: Wedderburn Decomposition into 289560 orbits and 4 simple summands of sizes
└ [590, 549, 484, 450]


In [63]:
@time model, varP = PropertyT.sos_problem_primal(
    AdjC₂,
    Δ,
    wd;
    augmented = true,
);
model

 48.033685 seconds (204.17 M allocations: 122.399 GiB, 9.01% gc time)


A JuMP Model
Maximization problem with:
Variables: 544166
Objective function type: JuMP.VariableRef
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 289559 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ

## Solving the problem numerically
We will use `scs` [Splitting Conic Solver](https://github.com/cvxgrp/scs) so solve this problem.

In [64]:
using MKL_jll
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));
with_optimizer = scs_optimizer(;
    linear_solver = SCS.MKLDirectSolver,
    eps = 1e-9,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.95,
);

In [65]:
warm = nothing

> **Note** 
> * To obtain just any positive lower bound it is advisable to (artificially) bound the objective from above, e.g. by adding
    ```julia
    JuMP.@constraint(model, upper_bound, model[:λ] ≤ 0.125)
    ```
    before solving the model (to bring the solve time to below 1h).
> * If you do not bound the objective you will need to re-run the cell below several times to obtain `status = OPTIMAL::TerminationStatusCode = 1`. To succesfully certify **a lower bound** that might not be necessary. However this will be necessary to obtain **the bound advertised** in the paper.


In [66]:
JuMP.@constraint(model, upper_bound, model[:λ] ≤ 0.125)

upper_bound : λ ≤ 0.125

In [67]:
status, warm = PropertyT.solve(
        model,
        with_optimizer,
        warm,
    );
# note: since we're using scs there will be no printout until the optimization has finished 
# please bear with us...
@info "Optimization has finished with" status

------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 544166, constraints m: 833725
cones: 	  z: primal zero / dual free vars: 289559
	  l: linear vars: 1
	  s: psd vars: 544165, ssize: 4
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-07
	  alpha: 1.95, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 20000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 50, acceleration_interval: 10
lin-sys:  sparse-direct-mkl-pardiso
	  nnz(A): 4965179, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.40e+01  1.00e+00  2.03e+01 -1.02e+01  1.00e-01  1.48e+02 
   250| 2.90e-03  2.43e-04  3.47e-03 -1.26e-01  2.31e+0

┌ Info: Optimization has finished with
└   status = OPTIMAL::TerminationStatusCode = 1


### Reconstructing and certifying the solution

In [68]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

[ Info: reconstructing the solution


  1.321305 seconds (11.98 M allocations: 825.351 MiB, 10.68% gc time, 4.25% compilation time)


2073×2073 Matrix{Float64}:
 0.0   0.0           0.0           0.0          …   0.0           0.0
 0.0   0.53076       0.122576      0.127844         0.00062125   -6.15139e-5
 0.0   0.122576      0.53076       0.13139          0.00062125   -6.15139e-5
 0.0   0.127844      0.13139       0.527937         0.000852002  -0.000178643
 0.0   0.13139       0.127844      0.129358         0.000852002  -0.000178643
 0.0   0.127784      0.127784      0.131408     …  -2.42835e-5   -0.000543139
 0.0   0.13139       0.127844      0.12953          0.0010291     0.00106489
 0.0   0.127844      0.13139       0.132646         0.0010291     0.00106489
 0.0   0.127784      0.127784      0.127877         0.000569626   0.00228599
 0.0   0.119758      0.122304      0.127844         0.00062125   -6.15139e-5
 0.0   0.122304      0.119758      0.13139      …   0.00062125   -6.15139e-5
 0.0   0.127844      0.13139       0.121956         0.000852002  -0.000178643
 0.0   0.13139       0.127844      0.132655         

In [69]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    AdjC₂,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Adj_C₂ is positive" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the positivity of Adj_C₂" certified λ
end

[ Info: certifying the solution


  0.166054 seconds (6 allocations: 41.550 MiB)


┌ Info: Checking in Float64 arithmetic with
└   λ = 0.12499999763821108
┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ≈ 6.27080334803982e-11
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ≈ 5.0405928043122056e-6
└  λ ≈ 0.12491934815334209


  1.452540 seconds (74 allocations: 804.395 MiB, 4.59% gc time)


┌ Info: Checking in Interval{Float64} arithmetic with
└   λ_int = [0.124999, 0.125]
┌ Info: Numerical metrics of the obtained SOS:
│ ɛ(elt - λu - ∑ξᵢ*ξᵢ) ∈ [-6.00861e-08, 6.00381e-08]
│ ‖elt - λu - ∑ξᵢ*ξᵢ‖₁ ∈ [4.98061e-06, 5.10075e-06]
└  λ ∈ [0.124918, 0.124921]
┌ Info: Certified result: Adj_C₂ is positive
└   PropertyT.IntervalArithmetic.inf(λ) = 0.12491838579643448


In [70]:
using Dates
Dates.now()

2023-06-22T15:25:38.024

In [71]:
versioninfo()

Julia Version 1.9.1
Commit 147bdf428cd (2023-06-07 08:27 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 PRO 4750U with Radeon Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, znver2)
  Threads: 9 on 16 virtual cores
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_IMAGE_THREADS = 1
